Goal: Usind data about strokes, build a predictor of which conditions are most likely to result in a sroke.

In [1]:
from config import CONFIG
import gradio as gr
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import classes.BMICalculator as bmic
import classes.Encoders as enc
# import classes.Formatter as fm

### EDA

In [2]:
# EDA
data_loca= CONFIG["DATA_FILEPATH"]
child_bmi_data = CONFIG["CHILD_BMI"]
data = pd.read_csv(data_loca)
child_bmi_df = pd.read_csv(child_bmi_data)
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [3]:
len(data)

5110

In [4]:
data_df = data.dropna()
len(data_df)

4909

In [5]:
# round ages to nearest int
data_df["age"] = data_df["age"].round()

/var/folders/18/918s5vr16kl9091zgnswr5gm0000gn/T/ipykernel_4696/4176842324.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["age"] = data_df["age"].round()


In [6]:
# can't calc bmi for ages below 2
data_df = data_df[data_df["age"] > 1]

In [7]:
len(data_df)

4828

In [8]:
# # get histograms of all data to see spread
# import matplotlib.pyplot as plt
# cols = ["gender", "age", "hypertension", "heart_disease","ever_married", "work_type","Residence_type", "avg_glucose_level","bmi","smoking_status","stroke"]
# for col in cols:
#     # counts = data_df[col].value_counts().plot.barh()
#     # fig, ax = plt.subplots()
#     # ax.set_title(f"{col}")
#     # plt.show()

#     categories = data_df[col].value_counts().index
#     counts = data_df[col].value_counts().values
#     plt.barh(categories, counts)
#     plt.title(col)
#     plt.show()


In [9]:
#  look at correlations
# import numpy as np
# import pandas as pd
# from scipy.stats import chi2_contingency

# def cramers_v(x, y):
#     confusion_matrix = pd.crosstab(x, y)
#     chi2 = chi2_contingency(confusion_matrix)[0]
#     n = confusion_matrix.sum().sum()
#     phi2 = chi2 / n
#     r, k = confusion_matrix.shape
#     phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
#     r_corr = r - ((r - 1) ** 2) / (n - 1)
#     k_corr = k - ((k - 1) ** 2) / (n - 1)
#     return np.sqrt(phi2corr / min((k_corr - 1), (r_corr - 1)))

# def plot_categorical_correlation(features):
#     num_features = features.shape[1]
#     correlation_matrix = np.zeros((num_features, num_features))

#     for i in range(num_features):
#         for j in range(num_features):
#             correlation_matrix[i, j] = cramers_v(features[:, i], features[:, j])

#     # Create a heatmap to visualize the correlation matrix
#     fig, ax = plt.subplots(figsize=(8, 8))
#     cax = ax.matshow(correlation_matrix, cmap='coolwarm', vmin=0, vmax=1)
#     plt.colorbar(cax)

#     # Add labels to the heatmap
#     feature_names = [f"Feature {i+1}" for i in range(num_features)]
#     ax.set_xticks(np.arange(len(feature_names)))
#     ax.set_yticks(np.arange(len(feature_names)))
#     ax.set_xticklabels(feature_names)
#     ax.set_yticklabels(feature_names)
#     ax.xaxis.tick_bottom()

#     # Rotate the x-axis labels for better visibility
#     plt.xticks(rotation=45, ha='right')

#     # Display the correlation values in each cell
#     for i in range(num_features):
#         for j in range(num_features):
#             ax.text(j, i, f"{correlation_matrix[i, j]:.2f}", ha='center', va='center', color='black')

#     plt.title("Categorical Correlation Matrix (Cramer's V)")
#     plt.xlabel("Features")
#     plt.ylabel("Features")
#     plt.show()


# plot_categorical_correlation(data_df)


In [10]:

# def plot_correlation_matrix(features):
#     # Calculate the correlation matrix using numpy.corrcoef
#     correlation_matrix = np.corrcoef(features, rowvar=False)

#     # Create a heatmap to visualize the correlation matrix
#     fig, ax = plt.subplots(figsize=(8, 8))
#     cax = ax.matshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
#     plt.colorbar(cax)

#     # Add labels to the heatmap
#     feature_names = [f"Feature {i+1}" for i in range(features.shape[1])]
#     ax.set_xticks(np.arange(len(feature_names)))
#     ax.set_yticks(np.arange(len(feature_names)))
#     ax.set_xticklabels(feature_names)
#     ax.set_yticklabels(feature_names)
#     ax.xaxis.tick_bottom()

#     # Rotate the x-axis labels for better visibility
#     plt.xticks(rotation=45, ha='right')

#     # Display the correlation values in each cell
#     for i in range(len(feature_names)):
#         for j in range(len(feature_names)):
#             ax.text(j, i, f"{correlation_matrix[i, j]:.2f}", ha='center', va='center', color='black')

#     plt.title("Pearson's Correlation Matrix")
#     plt.xlabel("Features")
#     plt.ylabel("Features")
#     plt.show()

# plot_correlation_matrix(data_df)


### BMI conversion data

These files contain the L, M, and S parameters needed to generate exact percentiles and z-scores along with the percentile values for the 3rd, 5th, 10th, 25th, 50th, 75th, 90th, 95th, and 97th percentiles by sex (1=male; 2=female) and single month of age. The smoothed 85th percentile values are included in the BMI-for-age and weight-for-stature tables. Age is listed at the half month point for the entire month; for example, 1.5 months represents 1.0-1.99 months or 1.0 month up to but not including 2.0 months of age. The only exception is birth, which represents the point at birth. To obtain L, M, and S values at finer age or length/stature intervals interpolation could be used.

The LMS parameters are the median (M), the generalized coefficient of variation (S), and the power in the Box-Cox transformation (L). To obtain the value (X) of a given physical measurement at a particular z-score or percentile, use the following equation:

X = M (1 + LSZ)**(1/L), L ≠ 0

Or

X = M exp(SZ), L = 0

where the L, M, and S are the values from the appropriate table corresponding to the age in months of the child (** indicates an exponent, such that M(1+LSZ)**(1/L) means raising (1+LSZ) to the (1/L)th power and then multiplying the M; exp(X) is the exponentiation function, e to the power X). Z is the z-score that corresponds to the percentile. z-scores correspond exactly to percentiles, e.g., z-scores of -1.881, -1.645, -1.282, -0.674, 0, 0.674, 1.036, 1.282, 1.645, and 1.881 correspond to the 3rd, 5th, 10th, 25th, 50th, 75th, 85th, 90th, 95th, and 97th percentiles, respectively.

For example, to obtain the 5th percentile of weight-for-age for a 9-month-old male, we would look up the L, M and S values from the WTAGEINF table, which are L=-0.1600954, M=9.476500305, and S=0.11218624. For the 5th percentile, we would use Z=-1.645. Using the equation above, we calculate that the 5th percentile is 7.90 kg.

To obtain the z-score (Z) and corresponding percentile for a given measurement (X), use the following equation:

((X/M)**L) – 1
Z = ————————-, L≠0
LS

or

Z = ln(X/M)/S ,L=0

where X is the physical measurement (e.g. weight, length, head circumference, stature or calculated BMI value) and L, M and S are the values from the appropriate table corresponding to the age in months of the child (or length/stature). (X/M)**L means raising the quantity (X/M) to the Lth power.

For example, to obtain the weight-for-age z-score of a 9-month-old male who weighs 9.7 kg, we would look up the L, M and S values from the WTAGEINF table, which are L=-0.1600954, M=9.476500305, and S=0.11218624. Using the equation above, we calculate that the z-score for this child is 0.207. This z-score corresponds to the 58th percentile.

Z-scores and corresponding percentiles can be obtained from standard normal distribution tables found in statistics text books. Standard normal tables can also be found on the internet by doing a search on a “standard normal table.” In addition, many computer programs have pre-existing functions that convert Z-scores to percentiles and vice versa.



In [11]:
# chile BMI conversion df
child_bmi_df.head()

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
0,1,24,-2.01118107,16.57502768,0.080592465,14.52095333,14.73731947,15.09032827,15.74164233,16.57502768,17.55718781,18.16219473,18.60948128,19.33801062,19.85985812
1,1,24.5,-1.982373595,16.54777487,0.080127429,14.50347667,14.71929257,15.07117474,15.71962876,16.54777487,17.52129279,18.11954923,18.56110634,19.27889813,19.79194014
2,1,25.5,-1.924100169,16.49442763,0.079233994,14.46882381,14.68360841,15.03335725,15.67634464,16.49442763,17.45135039,18.03668013,18.46729593,19.16465965,19.66102345
3,1,26.5,-1.86549793,16.44259552,0.078389356,14.43459737,14.64843329,14.99619505,15.634035,16.44259552,17.38383656,17.95700228,18.37736191,19.05567423,19.5365754
4,1,27.5,-1.807261899,16.3922434,0.077593501,14.40082828,14.61378626,14.95969047,15.5926798,16.3922434,17.3187102,17.88047101,18.29125345,18.9518675,19.41848805


In [12]:
child_bmi_df.describe()

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
count,439,439,439,439,439,439,439,439,439,439,439,439,439,439,439
unique,3,220,439,439,439,439,439,439,439,439,439,439,439,439,439
top,1,24,-2.01118107,16.57502768,0.080592465,14.52095333,14.73731947,15.09032827,15.74164233,16.57502768,17.55718781,18.16219473,18.60948128,19.33801062,19.85985812
freq,219,2,1,1,1,1,1,1,1,1,1,1,1,1,1


In [15]:
# # drop secondary heading (from joint male and female bmi chart)
child_bmi_df = child_bmi_df.drop(index = 219)

In [16]:
child_bmi_df["Sex"].unique()

array(['1', '2'], dtype=object)

In [17]:
child_bmi_df.dtypes

Sex       object
Agemos    object
L         object
M         object
S         object
P3        object
P5        object
P10       object
P25       object
P50       object
P75       object
P85       object
P90       object
P95       object
P97       object
dtype: object

In [18]:
child_bmi_df = child_bmi_df.astype(float)
child_bmi_df["Sex"] = child_bmi_df["Sex"].astype(int)

In [19]:
child_bmi_df.dtypes

Sex         int64
Agemos    float64
L         float64
M         float64
S         float64
P3        float64
P5        float64
P10       float64
P25       float64
P50       float64
P75       float64
P85       float64
P90       float64
P95       float64
P97       float64
dtype: object

In [20]:
child_bmi_df

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
0,1,24.0,-2.011181,16.575028,0.080592,14.520953,14.737319,15.090328,15.741642,16.575028,17.557188,18.162195,18.609481,19.338011,19.859858
1,1,24.5,-1.982374,16.547775,0.080127,14.503477,14.719293,15.071175,15.719629,16.547775,17.521293,18.119549,18.561106,19.278898,19.791940
2,1,25.5,-1.924100,16.494428,0.079234,14.468824,14.683608,15.033357,15.676345,16.494428,17.451350,18.036680,18.467296,19.164660,19.661023
3,1,26.5,-1.865498,16.442596,0.078389,14.434597,14.648433,14.996195,15.634035,16.442596,17.383837,17.957002,18.377362,19.055674,19.536575
4,1,27.5,-1.807262,16.392243,0.077594,14.400828,14.613786,14.959690,15.592680,16.392243,17.318710,17.880471,18.291253,18.951867,19.418488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,2,237.5,-2.353501,21.689489,0.151755,17.435152,17.822557,18.480142,19.788872,21.689489,24.384258,26.397231,28.130338,31.596640,34.818095
435,2,238.5,-2.350529,21.701083,0.152221,17.434275,17.822591,18.481819,19.794202,21.701083,24.406856,26.429839,28.172905,31.663237,34.912504
436,2,239.5,-2.346962,21.711892,0.152716,17.431988,17.821267,18.482230,19.798457,21.711892,24.429104,26.462429,28.215745,31.730686,35.008306
437,2,240.0,-2.344958,21.716999,0.152975,17.430311,17.820090,18.481956,19.800176,21.716999,24.440096,26.478720,28.237271,31.764743,35.056751


In [21]:
# child_bmi_df

t_age = 168.5
t_gender = 2
t_age_mask = child_bmi_df["Agemos"] == t_age
t_gender_mask = child_bmi_df["Sex"] == t_gender
            
t_lookup_row = child_bmi_df[t_age_mask & t_gender_mask]
t_lookup_row

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
365,2,168.5,-1.977074,19.352572,0.151256,15.442351,15.807529,16.423776,17.635089,19.352572,21.688194,23.34689,24.711407,27.255974,29.396856


## Encode data for training

In [22]:
# split data by age 
child_df = data_df[data_df["age"] <= 20]
adult_df = data_df[data_df["age"] > 20]

In [23]:
# encode bmi for each df
enc_child_bmi =  enc.ChildBMIEncoder(bmi_table= child_bmi_df, data_table= child_df).encode()
enc_adult_bmi = enc.AdultBMIEncoder(adult_df).encode()

/Users/jadeevans/Documents/Code/machine-learning/biomed-datasci-projs/stroke-prediction/classes/Encoders.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data_table['bmi_buckets'] = bmi_buckets
/Users/jadeevans/Documents/Code/machine-learning/biomed-datasci-projs/stroke-prediction/classes/Encoders.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data_table['bmi_buckets'] = bmi_buckets


In [22]:
# def combine_dfs(df1,df2):
#     if len(enc_child_bmi.columns) == len(enc_adult_bmi.columns):
#         new_df = pd.concat([enc_child_bmi,enc_adult_bmi])

#     else:
#         print("Columns must match to concatenate dataframes")
#         return new_df

In [24]:
if len(enc_child_bmi.columns) == len(enc_adult_bmi.columns):
    new_df = pd.concat([enc_child_bmi,enc_adult_bmi])

else:
    print("Columns must match to concatenate dataframes")

new_df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_buckets
245,49669,Female,14.0,0,0,No,children,Rural,57.93,30.9,Unknown,1,obese
249,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,Unknown,0,over weight
251,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,Unknown,0,healthy weight
253,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,Unknown,0,healthy weight
265,19584,Female,20.0,0,0,No,Private,Urban,84.62,19.7,smokes,0,healthy weight
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,45010,Female,57.0,0,0,Yes,Private,Rural,77.93,21.7,never smoked,0,healthy weight
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0,obese
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0,obese
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0,over weight


In [25]:
age_enc = enc.AgeEncoder(new_df,"age")
new_df = age_enc.encode()
# new_df = encode_age.df()
new_df
# encode_age


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,...,bmi_buckets,a0,a10,a20,a30,a40,a50,a60,a70,a80
245,49669,Female,14.0,0,0,No,children,Rural,57.93,30.9,...,obese,0,1,0,0,0,0,0,0,0
249,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,...,over weight,1,0,0,0,0,0,0,0,0
251,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,...,healthy weight,1,0,0,0,0,0,0,0,0
253,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,...,healthy weight,0,1,0,0,0,0,0,0,0
265,19584,Female,20.0,0,0,No,Private,Urban,84.62,19.7,...,healthy weight,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,45010,Female,57.0,0,0,Yes,Private,Rural,77.93,21.7,...,healthy weight,0,0,0,0,0,1,0,0,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,...,obese,0,0,0,0,0,0,0,0,1
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,...,obese,0,0,0,1,0,0,0,0,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,...,over weight,0,0,0,0,0,1,0,0,0


In [25]:
de.encode_bmi(new_df,"bmi_buckets")

new_df

NameError: name 'de' is not defined

In [26]:
gend_enc = enc.GenderEncoder(new_df,"gender").encode()
marr_enc = enc.MarriageEncoder(new_df,"ever_married").encode()
res_enc = enc.ResidenceEncoder(new_df,"Residence_type").encode()
smoke_enc = enc.SmokingStatusEncoder(new_df, "smoking_status").encode()

new_df = smoke_enc
new_df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,...,a60,a70,a80,is_male,c_ever_married,live_urban,ss_1,ss_2,ss_3,ss_4
245,49669,Female,14.0,0,0,No,children,Rural,57.93,30.9,...,0,0,0,0,0,0,1,0,0,0
249,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,...,0,0,0,1,0,0,1,0,0,0
251,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,...,0,0,0,0,0,1,1,0,0,0
253,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,...,0,0,0,1,0,0,1,0,0,0
265,19584,Female,20.0,0,0,No,Private,Urban,84.62,19.7,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,45010,Female,57.0,0,0,Yes,Private,Rural,77.93,21.7,...,0,0,0,0,1,0,0,0,1,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,...,0,0,1,0,1,1,0,0,1,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,...,0,0,0,0,1,0,0,0,1,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,...,0,0,0,1,1,0,0,0,0,1


In [ ]:
    # de.encode_gender(new_df)
    # de.encode_marriage(new_df)
    # de.encode_residence(new_df)
    # de.encode_smoke_stat(new_df,"smoking_status")
    # new_df

In [27]:
new_df["stroke"]

245     1
249     0
251     0
253     0
265     0
       ..
5102    0
5106    0
5107    0
5108    0
5109    0
Name: stroke, Length: 4828, dtype: int64

In [28]:
def get_train_df(df, drop_cols, label_col):
    """
    Input: 
        df with encoded training data
        [drop_cols]; columns names to drop from df
        "label_col"; name of label column
    Output: 
        training data df, label df
    """
    labels = df[label_col]
    df = df.drop(label_col, axis = 1)
    train_df = df.drop(drop_cols, axis = 1)

    return train_df, labels

In [29]:
# get train df, remove unecceasry columns
# removed avg glucose level for now (for demo)
# drop_cols = ["id", "gender", "age", "ever_married", "work_type", "Residence_type", "bmi","smoking_status", "bmi_buckets","avg_glucose_level"]
drop_cols = ["id", "gender", "ever_married", "work_type", "Residence_type", "bmi","smoking_status", "bmi_buckets","avg_glucose_level", "a0", "a10", "a20", "a30", "a40", "a50", "a60", "a70", "a80"]
train_df, labels = get_train_df(new_df, drop_cols, "stroke")
train_df

,age,hypertension,heart_disease,is_male,c_ever_married,live_urban,ss_1,ss_2,ss_3,ss_4
245,14.0,0,0,0,0,0,1,0,0,0
249,3.0,0,0,1,0,0,1,0,0,0
251,8.0,0,0,0,0,1,1,0,0,0
253,14.0,0,0,1,0,0,1,0,0,0
265,20.0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
5102,57.0,0,0,0,1,0,0,0,1,0
5106,81.0,0,0,0,1,1,0,0,1,0
5107,35.0,0,0,0,1,0,0,0,1,0
5108,51.0,0,0,1,1,0,0,0,0,1


In [30]:
train_df.columns

Index(['age', 'hypertension', 'heart_disease', 'is_male', 'c_ever_married',
       'live_urban', 'ss_1', 'ss_2', 'ss_3', 'ss_4'],
      dtype='object')

In [31]:
labels

245     1
249     0
251     0
253     0
265     0
       ..
5102    0
5106    0
5107    0
5108    0
5109    0
Name: stroke, Length: 4828, dtype: int64

In [32]:
# split into testing and training data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df, labels, test_size = 0.2, random_state = 42)

In [33]:
X_train.dtypes

age               float64
hypertension        int64
heart_disease       int64
is_male             int64
c_ever_married      int64
live_urban          int64
ss_1                int64
ss_2                int64
ss_3                int64
ss_4                int64
dtype: object

In [34]:
# use SVM for classification
from sklearn.svm import LinearSVC

classifier = LinearSVC()
classifier.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [35]:
predictions = classifier.predict(X_test)

In [36]:
classifier.score(X_test,y_test)

0.9585921325051759

In [37]:
classifier.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

In [ ]:
classifier.predict_proba(X_test)

AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

In [ ]:
# more metrics
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

In [ ]:

classification_report(y_test,predictions)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

'              precision    recall  f1-score   support\n\n           0       0.96      1.00      0.98       926\n           1       0.00      0.00      0.00        40\n\n    accuracy                           0.96       966\n   macro avg       0.48      0.50      0.49       966\nweighted avg       0.92      0.96      0.94       966\n'

Try to rerain using k-folds

In [ ]:
# new test-train splits
X_train2, X_test2, y_train2, y_test2 = train_test_split(train_df, labels, test_size = 0.3, random_state = 12)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

#define cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

model = classifier

#use k-fold CV to evaluate model
scores = cross_val_score(model, X_train2, y_train2, scoring=make_scorer(accuracy_score),
                         cv=cv, n_jobs=-1)

scores

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increas

array([0.57988166, 0.96153846, 0.9408284 , 0.9556213 , 0.96449704,
       0.95857988, 0.9408284 , 0.94674556, 0.95857988, 0.95548961])

In [ ]:
c2 = classifier
c2.fit(X_train2,y_train2)
pred2 = c2.predict(X_test2)
c2.score(X_test2,y_test2)



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.968944099378882

## DEMO

In [ ]:
# demo support functions

def request_age(age):
    return age

def encode_age(df):
    df.Age = df.Age.fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    categories = pd.cut(df.Age, bins, labels=False)
    df.Age = categories
    return df

def request_sex(sex):
    return "Male" if sex.lower() == "male" else "Female"

def hypertension(text):
    feature = 1 if text.lower() == "yes" else 0
    return feature

def heart_disease(text):
    feature = 1 if text.lower() == "yes" else 0
    return feature

    

def bmi_standard(weight_lb,height_ft_in, age):
    """ 
    Inputs:
        height_ft_in = (height feet, height in)
        weight_lb = 
    """
    height = (height_ft_in[0] * 12) + height_ft_in[1]
    bmi = weight_lb / (height **2)

    return bmi

    

def bmi_imperial(weight_kg, height_m,age):

    bmi = weight_kg / (height_m **2)

    return bmi

def req_marriage(text):
    if text.lower() == "yes":
        feature = "Yes"
    else: 
        feature = "No"
    return feature

def red_residence(text):
    if text.lower()== 

In [ ]:
def bmi_standard(weight_lb,height_ft_in, age):
    """ 
    Inputs:
        height_ft_in = (height feet, height in)
        weight_lb = 
    """
    height = (height_ft_in[0] * 12) + height_ft_in[1]
    bmi = weight_lb / (height **2)

    return bmi

    

def bmi_imperial(weight_kg, height_m,age):

    bmi = weight_kg / (height_m **2)

    return bmi

In [ ]:
# dropdown data
gr.CheckboxGroup(["Male", "Female"], label="Gender?")
gr.Dropdown(label = "Residence Type",choices =["Urban", "Rural"],show_label = True, interactive = True)

gr.Dropdown(label = "Smoking Status",choices = ["smokes","never smoked", "formerly smoked", "Unknown"],show_label = True)

gr.CheckboxGroup(["Urban", "Rural"], label="Residence Type")
# convert to 1 or 0 
gr.CheckboxGroup(["Yes", "No"], label="Hypertension?")
gr.CheckboxGroup(["Yes", "No"], label="Heart Disease?")
gr.Textbox("Age")

In [ ]:
# get units of height and weight
# get standard height as (feet,inches)

'age', 'hypertension', 'heart_disease', 'stroke', 'bmi_under',
       'bmi_healthy', 'bmi_over', 'bmi_obese', 'is_male', 'c_ever_married',
       'live_urban', 'ss_1', 'ss_2', 'ss_3', 'ss_4'
 

In [ ]:
# make an encoder interface (assembly line)

In [ ]:
# make enocde classes (for each function)


In [ ]:
# create a predict dict class and initialize with the object

In [ ]:
# example:


prediction_dict = {
                    "height": None, 
                    "height_unit": None, 
                    "weight": None, 
                    "weight_unit": None, 
                    "gender": None, 
                    "age": None, 
                    "marriage_stat": None, 
                    "residence": None, 
                    "smoke_stat": None, 
                    "hypertension": None,
                    "heart_disease": None
                }

def predict_survival(prediction_dict):
    """ 
    Input: 
        {prediction_dict};
            "height": None, 
            "height_unit": None, 
            "weight": None, 
            "weight_unit": None, 
            "gender": None, 
            "age": None, 
            "marriage_stat": None, 
            "residence": None, 
            "smoke_stat": None, 
            "hypertension": None,
            "heart_disease": None
    Output:
        Preddicton of stroke (Yes or No)
    """

  prediction_dict["height"] = 
        ["height_unit"]
        ["weight"]
        "weight_unit"
        "gender"
        "age"
        "marriage_stat"
        "residence"
        "smoke_stat"
        "hypertension"
        "heart_disease"
    if height_unit == "kg" and weight_unit == "m":
        bmi = bmi_imperial(weight_kg= weight, height_m= height, age= age)
    else:
        bmi = bmi_standard(weight_lb= weight, height_ft_in=height, age = age)
    
    # initialize df
    df = pd.DataFrame.from_dict(
        {
            "age": 0,
            "gender": gender
            "hypertension": 0,
            "heart_disease": 0,
            "bmi" = bmi
            "live_urban": 0,
            "ever_married": marriage_stat,
            "Residence_type": residence,
            "smoking_status": smoke_stat
                    }
    )
    df = de.get_bmi_bucket(data_table = data_df, bmi_table =child_bmi_df )
    df = de.encode_bmi(df,"bmi_buckets")
    # df = de.encode_ages(df,"age")
    df = de.encode_gender (df)
    df = de.encode_marriage(df)
    df = de.encode_residence(df)
    df = de.encode_smoke_stat(df,"smoking_status")

    drop_cols = ["gender", "bmi"]

    pred = classifier.predict(df)
    return {"Stroke likely": float(pred[0]), "Survives": float(pred[1])}


demo = gr.Interface(
    predict_survival,
    [
        gr.Dropdown(["first", "second", "third"], type="index"),
        "checkbox",
        gr.Slider(0, 80, value=25),
        gr.CheckboxGroup(["Sibling", "Child"], label="Travelling with (select all)"),
        gr.Number(value=20),
        gr.Radio(["S", "C", "Q"], type="index"),
    ],
    "label",
    examples=[
        ["first", True, 30, [], 50, "S"],
        ["second", False, 40, ["Sibling", "Child"], 10, "Q"],
        ["third", True, 30, ["Child"], 20, "S"],
    ],
    interpretation="default",
    live=True,
)

if __name__ == "__main__":
    demo.launch()

In [65]:
import gradio as gr
from classes.BMICalculator import BMICalculator


def greet(name):
    return "Hello " + name + "!"



# def predict_stroke(data):
#     """ 
#     hypertension (y/n)
# heart disease (y/n)
# avg glucose level (?)
# bmi (sub for weight and calculate?)
# ever married (y/n/child)
# age 
# neighborhood (urban, rural)
# gender
# smoking status (former, never, yes,  )
# """
#     if passenger_class is None or embark_point is None:
#         return None
#     df = pd.DataFrame.from_dict(
        



#     )
    
# demo = gr.Interface(fn=greet, inputs="text", outputs="text")
def get_data(data_dict):
    data_dict[age_in]


    # make into df
    df = pd.DataFrame.from_dict(data_dict)

    # make fcn to encode data
    return pred

im = gr.PlayableVideo(value= "/Users/jadeevans/Documents/Code/machine-learning/biomed-datasci-projs/stroke-prediction/data/notdoc.mp4", autoplay = True)

def output_video():
    im = gr.PlayableVideo(value= "/Users/jadeevans/Documents/Code/machine-learning/biomed-datasci-projs/stroke-prediction/data/notdoc.mp4", autoplay = True)
    return im

data = {}
age_in = gr.Textbox(label = "Age"),
gend_in = gr.CheckboxGroup(["Male", "Female"], label="Gender?")
res_in = gr.CheckboxGroup(["Urban", "Rural"], label="Residence Type")
hyp_in = gr.CheckboxGroup(["Yes", "No"], label="Hypertension?")
hdis_in = gr.CheckboxGroup(["Yes", "No"], label="Heart Disease?")
smoke_in = gr.CheckboxGroup(label = "Smoking Status",choices = ["smokes","never smoked", "formerly smoked", "Unknown"],show_label = True)
bmi_comp = gr.CheckboxGroup(label = "For BMI Calculation", choices = [
                                                                        gr.Textbox(label = "Height"),
                                                                        gr.CheckboxGroup(["Feet , Inches (ft , in)", "Meters (m)"], label="Height units"),
                                                                        gr.Textbox(label = "Weight"),
                                                                        gr.CheckboxGroup(["Pounds (lb)", "Kilograms (kg)"], label="Weight units")
])
req_bmi = BMICalculator(data_dict)
# gr.Dropdown(label = "Residence Type",choices =["Urban", "Rural"],show_label = True, interactive = True),
# gr.Dropdown(label = "Smoking Status",choices = ["smokes","never smoked", "formerly smoked", "Unknown"],show_label = True)



demo = gr.Interface(fn = get_data, inputs = data,
                    
                    
                     outputs = im, live = True)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7888
Running on public URL: https://1056482a055a6698f7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/routes.py", line 442, in run_predict
    output = await app.get_blocks().process_api(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1395, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1329, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/components/video.py", line 279, in postprocess
    processed_files = (self._format_video(y), None)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/components/video.py", line 344, in _format_video
    output_file_name = video[0 : video.rinde

In [ ]:
import gradio as gr
import os
import classes.BMICalculator as bmic

def video_identity(video):
    return video


with gr.Blocks(live=True) as demo:
    
    h_text = gr.Textbox(label = "Height")
    h_unit = gr.CheckboxGroup(["ft,in", "m"], label = "Height units")
    w_text = gr.Textbox(label = "Weight")
    w_unit = gr.CheckboxGroup(["lb", "kg"], label = "Weight units")

    def calc_bmi(data):
        height = data[h_text]
        height_unit = data[h_unit][0]
        weight = data[w_text]
        weight_unit = data[w_unit][0]

        error_message = ""

        len_height_unit = len(data[h_unit])
        len_weight_unit = len(data[w_unit])
        multiple_units_selected = any(i != 1 for i in [len_height_unit, len_weight_unit])

        if multiple_units_selected:
            error_message = "Please select one unit each for height and weight, then submit again."
            return error_message
        elif height_unit == "m" and weight_unit == "kg":
            imp_obj = bmic.ImperialBMICalculator(weight_kg = weight, height_m = height)
            bmi = imp_obj.calculate()
        elif height_unit == "ft,in" and weight_unit == "lb":
            imp_obj = bmic.StandardBMICalculator(weight_lb = weight, height_ft_in = height)
            bmi = imp_obj.calculate()
        else:
            bmi = None
        
        if not bmi: 
            error_message = "BMI input units need to be consitently imperial (kg and m) or standard (ft,in and lb). Please try again."
            return error_message
        
        return bmi
    
    output = gr.Text(label= "BMI")
    with gr.Row():
        add_btn = gr.Button("Submit")
    
    add_btn.click(calc_bmi, inputs={h_text, h_unit, w_text, w_unit}, outputs=output)
    # (calc_bmi, 
    #                 # gr.Video(value="/Users/jadeevans/Documents/Code/machine-learning/biomed-datasci-projs/stroke-prediction/data/notdoc.mp4"), 
    #                 # "playable_video")
    #                 {h_text, h_unit, w_text, w_unit},
                    
    #                 "text")

if __name__ == "__main__":
    demo.launch()

/var/folders/18/918s5vr16kl9091zgnswr5gm0000gn/T/ipykernel_5589/2842918480.py:9: GradioUnusedKwargWarning: You have unused kwarg parameters in Blocks, please remove them: {'live': True}
  with gr.Blocks(live=True) as demo:


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/routes.py", line 442, in run_predict
    output = await app.get_blocks().process_api(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1392, in process_api
    result = await self.call_function(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1097, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Library/Frameworks/Python.framework/Versions/3.10

Input must contain only numbers separated by ",". Ex: "5,2" for 5 feet, 2 inches


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/routes.py", line 442, in run_predict
    output = await app.get_blocks().process_api(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1392, in process_api
    result = await self.call_function(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1097, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Library/Frameworks/Python.framework/Versions/3.10

Input must contain only numbers separated by ",". Ex: "5,2" for 5 feet, 2 inches


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/routes.py", line 442, in run_predict
    output = await app.get_blocks().process_api(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1392, in process_api
    result = await self.call_function(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1097, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Library/Frameworks/Python.framework/Versions/3.10

In [5]:
any([True,True])

True